<a href="https://colab.research.google.com/github/Hemant2388/Sentiment_Analysis_using_llm_transfer_learning/blob/main/sentiment_analysis_llama_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

multi_lingual_sentiment_analysis_path = kagglehub.competition_download('multi-lingual-sentiment-analysis')
metaresearch_llama_3_1_transformers_8b_instruct_2_path = kagglehub.model_download('metaresearch/llama-3.1/Transformers/8b-instruct/2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
 !pip install bitsandbytes evaluate datasets

In [ ]:
import warnings
from pprint import pprint
import math
import wandb

#hf
import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import DataCollatorWithPadding
from transformers import LlamaConfig, LlamaForCausalLM,LlamaForSequenceClassification
from transformers import TrainingArguments, Trainer

In [ ]:
train_df = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/train.csv')
test_df = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')

In [ ]:

import torch

In [ ]:
label_map = {"Negative": 0, "Positive": 1}
train_df["label"] = train_df["label"].map(label_map)

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
ds = DatasetDict({"train": train_dataset, "test": test_dataset})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path,model_max_length=1024)
# set pad token id
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tokenize(example):
    example = tokenizer(example['sentence'],padding=False,truncation=True)
    return example

In [ ]:
ds

In [ ]:
tokenized_ds = ds.map(tokenize,batched=True,num_proc=12, remove_columns=['ID', 'sentence', 'language'])
print(tokenized_ds)

In [ ]:
ds_split = tokenized_ds['train'].train_test_split(test_size=0.1,seed=42)
print(ds_split)

In [ ]:
 data_collator = DataCollatorWithPadding(tokenizer,padding=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Best for LLaMA models
    bnb_4bit_compute_dtype=torch.float16,  # Can also use bfloat16 if supported
    bnb_4bit_use_double_quant=True,  # Further reduces memory usage
)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_path,
#     num_labels=2,
#     pad_token_id=tokenizer.eos_token_id,
#     quantization_config=bnb_config,
#     device_map="auto"
# )

In [ ]:
from peft import LoraConfig, TaskType, LoraModel
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none'
)

In [ ]:
# from peft import LoraConfig, get_peft_model, TaskType
# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()

In [ ]:
  # for name,param in model.named_parameters():
  #     if name != "score.weight":
  #        param.requires_grad = False
  #     print(name,param.requires_grad)


In [ ]:
#  num_parameters = 0
#  for param in model.parameters():
#      if param.requires_grad:
#          num_parameters += param.numel()
# print(f'Number of Parameters:{num_parameters}')

In [ ]:
import numpy as np
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import torch
print(torch.cuda.is_bf16_supported())

In [ ]:
#model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    output_dir='./llama8_multilingual_ft',
    evaluation_strategy="steps",
    eval_steps=10,
    num_train_epochs=3,  # Train for more epochs since data is limited
    per_device_train_batch_size=2,  # Lowered for memory reasons (P100 GPU)
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # Effective batch size is 2 * 8 = 16
    bf16=False,  # P100 doesn't support BF16
    fp16=False,  # Mixed precision training for memory efficiency
    gradient_checkpointing=True,  # Save memory on large models
    optim="adamw_torch",  # More efficient optimizer with torch backend
    learning_rate=0.1,  # Typically better for LoRA fine-tuning
    weight_decay=0.01,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,  # Save less frequently to avoid storage issues
    report_to='none',
    dataloader_pin_memory=True,
    dataloader_num_workers=2,  # Reduce to avoid Kaggle kernel crashes
    ddp_find_unused_parameters=False,  # Single GPU, this is safe
    max_grad_norm=1.0,
    warmup_steps=10,  # Helps stabilize early training
    lr_scheduler_type="cosine",  # Often works well with fine-tuning LLaMA
)


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=ds_split["train"],
#     eval_dataset=ds_split["test"],
#     data_collator = data_collator,
#     compute_metrics = compute_metrics
# )

In [ ]:
#results = trainer.train()

In [ ]:
#simple model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
print(f'Vocab size: {tokenizer.vocab_size}')
print(f'Context length: {tokenizer.model_max_length}')

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    num_labels = 2
)

In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )

In [ ]:
predictions = []

for idx, row in test_df.iterrows():
    prompt = prompt = f"""
Classify the sentiment of the provided sentence as either **"Positive"** or **"Negative"** based on its overall tone and context. The sentiment reflects the speaker's satisfaction, opinion, or emotional response.

Language: {row['language']}
Sentence: "{row['sentence']}"

Answer with only "Positive" or "Negative":
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=2)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)


    result_cleaned = result.strip().split()[-1].capitalize()# Handles cases like "Positive\nExplanation"

    if result_cleaned not in ["Positive", "Negative"]:
        result_cleaned = "Positive"  # Fallback or handle it differently

    if result_cleaned == "Positive":
        predictions.append("Positive")
    elif result_cleaned == "Negative":
        predictions.append("Negative")
    else:
        predictions.append("Positive")  # Fallback if it's neither

In [ ]:
(pd.DataFrame(predictions)).value_counts()

In [ ]:
submission = pd.DataFrame(columns = ['ID', 'label'])
submission['ID'] = [i for i in range(1, len(predictions)+1)]
submission['label'] = predictions
submission.to_csv('submission.csv', index=False)